In [1]:
import sys

sys.path.append("..")

from logger import get_logger

%load_ext autoreload
%autoreload 2

In [2]:
logger = get_logger(__name__)
logger.debug("This is a debug message")

DEBUG:__main__:This is a debug message
